# Trabalho Prático 3  #
#### André Freitas PG54707 ####
#### Bruna Macieira PG54467 ####

## Exercício 1

No capítulo 5 dos apontamentos é descrito o chamado Hidden Number Problem. No capítulo 8 dos apontamentos é discutida um artigo de  Nguyen & Shparlinsk , onde se propõem reduções do HNP a problemas difíceis em reticulados. Neste trabalho pretende-se construir, com a ajuda do Sagemath, uma implementação da solução discutida nos apontamentos para resolver o HNP com soluções aproximadas dos problemas em reticulados.

In [78]:
%pip install sagemath-standard
from sage.all import *
import copy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Etapa 1: Construção da matriz 𝐺.

In [79]:
def build_G_matrix(p, x_values, A, u_values, M, B):
    n = len(x_values)
    m = n + 2
    G = Matrix(QQ, m, m)

    # Preenchendo os elementos de G
    for i in range(m):
        for j in range(m):
            if i < n and j < n:
                G[i, j] = p if i == j else 0
            elif i == n:
                G[i, j] = x_values[j] if j < n else A if j == n else 0
            elif i == n + 1:
                G[i, j] = -B * u_values[j] if j < n else 0 if j == n else M
    print("G:", G)

    return G

Etapa 2: Construção do vetor alvo 𝑦.

In [80]:
def build_y_vector(u_values, B, M):
    n = len(u_values)
    y = [B * u for u in u_values] + [0, M]  # Construindo o vetor y
    print("y:", vector(y))

    return vector(y)

Etapa 3: Resolução do Problema do Menor Vetor em uma Reticulado (BDD).

In [81]:
# Função para resolver o BDD
def solve_BDD(G, y, B):
    # Construindo o reticulado e reduzindo a base
    L = Matrix(G).LLL()

    # Convertendo y para um vetor de SageMath
    y_vector = vector(y)

    # Calculando o ponto mais próximo de y no reticulado
    closest_point = L * y_vector

    # Arredondando o ponto mais próximo
    rounded_point = vector(ZZ, [round(coord) for coord in closest_point])

    print("G:", Matrix(G))

    print("L:", L)
    print("y:", y_vector)  
    print("Ponto mais próximo:", closest_point)
    print("Ponto arredondado:", rounded_point)
    print("Distância:", vector(rounded_point - y_vector).norm())
    print("B:", B)
    # Verificando se a distância é menor que B
    if vector(rounded_point - y_vector).norm() <= B:
        return rounded_point
    else:
        return None

Etapa 4: Recuperação do segredo 𝑠 a partir do vetor solução.

In [82]:
def recover_secret(reduced_matrix, lambda_value):
    # Extraindo a última componente do vetor solução
    e_n_plus_1 = reduced_matrix[-1][-2]

    # Arredondando para obter o valor de s
    s = round(lambda_value * e_n_plus_1)

    return s

Etapa 5: Avaliação da precisão da solução.

In [83]:
# Função para avaliar a precisão da solução
def evaluate_solution(y, x_values, s, p, B):
    # Calculando o vetor obtido a partir do segredo recuperado
    y_obtained = vector([s * xi % p for xi in x_values] + [0, 0])

    # Calculando a distância entre y e y_obtained
    distance = vector(y - y_obtained).norm()

    # Verificando se a distância é menor ou igual a B
    if distance <= B:
        return True, distance
    else:
        return False, distance

Definir os parâmetros a usar para resolver

In [84]:
k = 8  # Definindo o valor de k
lambda_value = 2**k  # Calculando o valor de lambda

# Parâmetros do problema
p = next_prime(1457)  # Primo p
x_values = [2, 1, 1, 2, 0]  # Valores xi
u_values = [1, 1, 0, 2, 3]  # Valores ui
A = 1/lambda_value  # Valor A é 1/lambda
B = p/lambda_value # Parâmetro B é p/lambda
M = M = 10^6  # Valor M

In [85]:
# Construindo a matriz G
G = build_G_matrix(p, x_values, A, u_values, M, B)

G: [     1459         0         0         0         0         0         0]
[        0      1459         0         0         0         0         0]
[        0         0      1459         0         0         0         0]
[        0         0         0      1459         0         0         0]
[        0         0         0         0      1459         0         0]
[        2         1         1         2         0     1/256         0]
[-1459/256 -1459/256         0 -1459/128 -4377/256         0   1000000]


In [86]:
# Construindo o vetor alvo y
y = build_y_vector(u_values, B, M)

y: (1459/256, 1459/256, 0, 1459/128, 4377/256, 0, 1000000)


In [87]:
# Resolvendo o BDD
solution = solve_BDD(G, y, B)
print("Solution:", solution)

G: [     1459         0         0         0         0         0         0]
[        0      1459         0         0         0         0         0]
[        0         0      1459         0         0         0         0]
[        0         0         0      1459         0         0         0]
[        0         0         0         0      1459         0         0]
[        2         1         1         2         0     1/256         0]
[-1459/256 -1459/256         0 -1459/128 -4377/256         0   1000000]
L: [        2         1         1         2         0     1/256         0]
[        0         0         0         0         0 -1459/256         0]
[      291      -584      -584       291         0    -73/32         0]
[      875      -292      -292      -584         0    -73/64         0]
[        1       730      -729         1         0  -729/256         0]
[        0         0         0         0      1459         0         0]
[  589/256  -435/256         4  -435/128 -4377/256      1/

In [88]:
# Verificando se a solução foi encontrada
if solution:
    # Recuperando o segredo
    s = recover_secret(solution, A)

    # Avaliando a precisão da solução
    is_within_limit, distance = evaluate_solution(y, x_values, s, p, B)

    # Exibindo resultados
    if is_within_limit:
        print("Solução encontrada dentro do limite B.")
        print("Segredo recuperado:", s)
        print("Distância entre y e y_obtained:", distance)
    else:
        print("Solução encontrada, mas está fora do limite B.")
        print("Segredo recuperado:", s)
        print("Distância entre y e y_obtained:", distance)
else:
    print("Não foi possível encontrar uma solução dentro do limite B.")

Não foi possível encontrar uma solução dentro do limite B.
